In [6]:
import numpy as np
import tensorflow as tf
import pathlib
import argparse
import json
import logging
import pathlib
from time import perf_counter

import tensorflow as tf
from tensorflow_graphics.geometry import transformation
from copy import deepcopy
from colorama import Fore
import pickle

from link_bot_data.dynamics_dataset import DynamicsDataset
from state_space_dynamics import model_utils
from moonshine.gpu_config import limit_gpu_mem
from link_bot_pycommon.pycommon import print_dict
from IPython.display import HTML
from link_bot_pycommon.get_scenario import get_scenario
from link_bot_pycommon.link_bot_scenario import LinkBotScenario
from moonshine.moonshine_utils import dict_of_sequences_to_sequence_of_dicts_tf, dict_of_sequences_to_sequence_of_dicts, numpify
import matplotlib.pyplot as plt
from link_bot_data.classifier_dataset_utils import *
from link_bot_classifiers import classifier_utils
from matplotlib import rc

np.set_printoptions(precision=3, suppress=True)
plt.style.use("slides")

limit_gpu_mem(2)

In [2]:
take = None
scenario = get_scenario("link_bot")
dynamics_dataset = DynamicsDataset([pathlib.Path("./fwd_model_data/rope_more_obs_big/")])
n_total_train_examples = 8250
tf_dataset = dynamics_dataset.get_datasets(mode='train', take=take)
model_dirs = [pathlib.Path(f"./ss_log_dir/tf2_rope/{i}") for i in range(8)]
fwd_models, _ = model_utils.load_generic_model(model_dirs)

classifier_model_dir = pathlib.Path('log_data/rope_2_seq/May_24_01-12-08_617a0bee2a')
classifier_model = classifier_utils.load_generic_model(classifier_model_dir, scenario=scenario)

W0602 11:48:52.127742 140634789559168 recurrent_v2.py:1102] Layer lstm will not use cuDNN kernel since it doesn't meet the cuDNN kernel criteria. It will use generic GPU kernel as fallback when running on GPU


Restored from ss_log_dir/tf2_rope/0/ckpt-40
Restored from ss_log_dir/tf2_rope/1/ckpt-44
Restored from ss_log_dir/tf2_rope/2/ckpt-45
Restored from ss_log_dir/tf2_rope/3/ckpt-43
Restored from ss_log_dir/tf2_rope/4/ckpt-40
Restored from ss_log_dir/tf2_rope/5/ckpt-38
Restored from ss_log_dir/tf2_rope/6/ckpt-40
Restored from ss_log_dir/tf2_rope/7/ckpt-49
Restored from log_data/rope_2_seq/May_24_01-12-08_617a0bee2a/ckpt-5


In [3]:
labeling_params = {
    'threshold': 0.10,
    'state_key': 'link_bot',
}

In [4]:
# tf.random.set_seed(0)
# transitions = []
# for example in tf_dataset:
#     inputs, outputs = example
#     for start_t  in range(0, dynamics_dataset.max_sequence_length - 2):
#         action = numpify(inputs['action'][start_t])
#         full_env = numpify(inputs['full_env/env'])
#         full_env_extent = numpify(inputs['full_env/extent'])
#         rope_state = numpify(outputs['link_bot'][start_t])
#         next_rope_state = numpify(outputs['link_bot'][start_t + 1])
        
#         transition = (rope_state,
#                       action,
#                       next_rope_state,
#                       full_env,
#                       full_env_extent)
#         transitions.append(transition)
transitions = pickle.load(open("transitions.pkl", 'rb'))

In [34]:
def close(t_1, t_2, threshold=0.02):
    next_rope_state_1 = LinkBotScenario.to_rope_local_frame_np(t_1[2])
    next_rope_state_2 = LinkBotScenario.to_rope_local_frame_np(t_2[2])
    is_close = np.linalg.norm(next_rope_state_2 - next_rope_state_1) < threshold 
    return is_close

In [35]:
transition_1 = transitions[0]
nearby_transitions = []
for transition_2 in transitions:
    if not np.all(transition_1[0] == transition_2[0]):
        if close(transition_1, transition_2):
            nearby_transitions.append(transition_2)

In [40]:
def state_from_transition(transition, local_frame=False):
    rope_state = transition[0]
    if local_frame:
        rope_state = LinkBotScenario.to_rope_local_frame_np(rope_state)
    return {
        'link_bot': rope_state,
    }

def next_state_from_transition(transition, local_frame=False):
    rope_state = transition[2]
    if local_frame:
        rope_state = LinkBotScenario.to_rope_local_frame_np(rope_state)
    return {
        'link_bot': rope_state,
    }

def action_from_transition(transition):
    return transition[1]

def env_from_transition(transition):
    return {
        'full_env/env': transition[3],
        'full_env/extent': transition[4]
    }

In [41]:
def plot_transition(ax, transition, color):
    state = state_from_transition(transition, local_frame=True)
    scenario.plot_state(ax, state, color=color, s=10, zorder=1)
    environment = env_from_transition(transition)
    scenario.plot_environment(ax, environment)
    scenario.plot_action(ax, state, action_from_transition(transition), color='c', s=10, zorder=1)

In [42]:
%matplotlib
plt.figure()
ax = plt.gca()
plt.axis("equal")

# plot_transition(ax, transition_1, 'r')
state = next_state_from_transition(transition_1, local_frame=True)
scenario.plot_state(ax, state, color='r', s=10, zorder=1)
for nearby_transition in nearby_transitions:
    state = next_state_from_transition(nearby_transition, local_frame=True)
    scenario.plot_state(ax, state, color='b', s=10, zorder=1, alpha=0.2)

Using matplotlib backend: Qt5Agg


In [39]:
len(nearby_transitions)

0